<a href="https://colab.research.google.com/github/astronights/mnist_gan/blob/main/MNIST_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Generative Adversarial Network to generate Handwritten Digits using MNIST data set
